In [1]:
import sqlite3
import numpy
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('training.db')
c = conn.cursor()

In [2]:
from dictionary import *
chars = []
for key,val in arpabetDict.items():
    chars += val
chars = list(set(chars))
char_dict = {chars[i].upper():i+1 for i in range(len(chars))}
char_dict["#"] = 0
arpabets = list(arpabetDict.keys())
# from IPython.core.debugger import Tracer; Tracer()() 
arp_dict = {arpabets[i].upper():i+1 for i in range(len(arpabets))}
arp_dict["#"]=0
# chars

In [3]:
all_ph = set()
for i in range(1,3+1):
    c.execute('SELECT DISTINCT "' + str(i) + '_before_arp" FROM Training_table ')
    all_ph.update(c.fetchall())
    c.execute('SELECT DISTINCT "' + str(i) + '_after_arp" FROM Training_table ')
    all_ph.update(c.fetchall())
all_ph = list(all_ph)
for i in range(len(all_ph)):
    all_ph[i] = all_ph[i][0]
all_ph = sorted(all_ph)
# print (all_ph)
c.execute('select DISTINCT character FROM Training_table ')
all_ch = sorted([x[0]for x in c.fetchall()])
# all_ch
# X

In [4]:
# c.execute('SELECT "1_before_arp","2_before_arp" ,"3_before_arp" ,"1_after_arp" ,"2_after_arp" ,"3_after_arp" ,  arpabet  FROM Training_table  ')
# x = c.fetchone()
# print len(features(x))

In [22]:

def one_hot(el,li):
    ret = [0]*(len(li)+1)
#     print(str(el) +","+ str(li))
#     from IPython.core.debugger import Tracer; Tracer()() 
    ret[li.index(el)] = 1
#     print ret
    return ret[1:]
    
def features(row):
    ret = []
    for x in row[:-1]:
        ret += one_hot(x,all_ph)
    ret.append(row[-1])
    return ret
#     return [
#         arp_dict[row[0]],
#         arp_dict[row[1]],
#         arp_dict[row[2]],
#         arp_dict[row[3]],
#         arp_dict[row[4]],
#         arp_dict[row[5]],
#         arp_dict[row[6]],
# #         arp_dict[row[7]],
# #         char_dict[row[7]]
#     ]
def generatorY():
    
    n = 0
    c.execute('select character FROM Training_table ')
    Y = c.fetchall()
    while n < len(Y):
        yield all_ch.index(Y[n][0])
        n+= 1
        
def generatorX():
    num = 0
    c.execute('SELECT count(*) FROM Training_table')
    total = c.fetchall()[0][0]
    while num < total:
        if num % 1000 == 0:
            c.execute('SELECT "1_before_arp","2_before_arp" ,"3_before_arp" ,\
          "1_after_arp" ,"2_after_arp" ,"3_after_arp" ,  arpabet  FROM Training_table LIMIT '+num+" , 1000")
            X = [features(row) for row in c.fetchall()]
            
        yield X[nuall_chm%1000]
        num += 1
        
def getX():
    num = 0
    c.execute('SELECT count(*) FROM Training_table')
    total = c.fetchall()[0][0]
    while num < total:
        if num % 1000 == 0:
            c.execute('SELECT "1_before_arp","2_before_arp" ,"3_before_arp" ,\
          "1_after_arp" ,"2_after_arp" ,"3_after_arp" ,  arpabet  FROM Training_table LIMIT '+num+" , 1000")
            X = [features(row) for row in c.fetchall()]
            
        X[nuall_chm%1000]
        num += 1
    return X

def getY():
    
    n = 0
    c.execute('select character FROM Training_table ')
    Y = c.fetchall()
    while n < len(Y):
        all_ch.index(Y[n][0])
        n+= 1
    return Y
    
#commenting this out because fit doesnot take generators:pFiT
# X = generatorX()
# Y = generatorY()
i = 0
# for y in Y:
#     print (y)
#     i+=1
#     if i==5:
#         break
c.execute('SELECT "1_before_arp","2_before_arp" ,"3_before_arp" ,\
          "1_after_arp" ,"2_after_arp" ,"3_after_arp" ,  arpabet , character FROM Training_table ORDER BY RANDOM() LIMIT 100000')
X = [features(row) for row in c.fetchall()]
# Y = getY()
Y = [x[-1] for x in X]
X = [x[:-1] for x in X]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [23]:
# from sklearn.preprocessing import OneHotEncoder
# import numpy
# encX = OneHotEncoder()
# encX.fit(X)
# _X = encX.transform(X).toarray()
# Y = numpy.matrix(Y)
# encY = OneHotEncoder()
# # Y = numpy.reshape(Y,len(Y),1)
# encY.fit(Y.T)
# _Y = encY.transform(Y.T).toarray()

In [24]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# model = RandomForestClassifier(n_jobs=-1)
model = GradientBoostingClassifier()
model.fit(X_train,Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [25]:
pred=model.predict(X_test)

In [26]:
preds = [chars[numpy.argmax(ch.tolist())] for ch in pred]

In [27]:
preds[0]

'oo'

In [28]:
chars[numpy.argmax(Y_test[0])]

'oo'

In [29]:
model.score(X_test,Y_test)

0.86419999999999997

In [30]:
importance_list = model.feature_importances_.tolist()

In [31]:
len(importance_list)

280

In [32]:
len(all_ph)

40

In [33]:
importance = []
for i in range(0, 7):
    importance+=[sum(importance_list[i*40:(i*40)+40])]
print(importance)

[0.17286694028283045, 0.11178494007456272, 0.08952373796228982, 0.20316342843519902, 0.10150566949060162, 0.05760530817700412, 0.24420571328243018]


In [34]:
#getting accuracy over training
pred_train = model.predict(X_train)


In [35]:
model.score(X_train,Y_train)

0.88822857142857148

In [36]:
importance_list = model.feature_importances_.tolist()

In [37]:
len(importance_list)

280

In [38]:
importance = []
for i in range(0, 7):
    importance+=[sum(importance_list[i*40:(i*40)+40])]
print(importance)

[0.17286694028283045, 0.11178494007456272, 0.08952373796228982, 0.20316342843519902, 0.10150566949060162, 0.05760530817700412, 0.24420571328243018]
